This is a simple modification to the Custom Search + Code Interpreter + File Search code that updates the output to something with nice formatting rather than the previous which had equations in pure text format of latex which was essentially unreadable. Everything in this is the same apart from one section of the event handler, scroll down to that part to see the explanation. (also remember to change your file_path when running this code)

In [1]:
#pip install openai

In [2]:
query = "how do i find the minimum chi squared value?"

In [ ]:
from openai import OpenAI
import openai 
import numpy as np

openai_api_key = ""

client = OpenAI(api_key=openai_api_key)

In [ ]:
assistant = client.beta.assistants.create(
  name="Discovery Skills",
  description="You are a factual education AI Assistant dedicated to providing accurate, useful information. Your primary task is to assist me by providing me reliable and clear responses to my questions, only ever use information from file search as your source, this knowledge base is _____.  You are reluctant of making any claims unless they are stated or supported by the knowledge base.",
  instructions = "if you make any code you should always run it",
  model="gpt-4o",
  tools=[{"type": "file_search"}, {"type": "code_interpreter"}],
  top_p=0.1,
)

In [10]:
import os
current_directory = os.getcwd()
print(current_directory)

d:\Documents\Summer_Project24\20-6-24 to 26-6-24\RAG + CI + FS Latex Formatted


In [ ]:
file_path = r"the above folder"

In [ ]:
chunks = []

def load_large_document(file_path, chunk_size=2000, encoding='utf-8'):
    with open(file_path, 'r', encoding=encoding) as file:
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break
            yield chunk

for chunk in load_large_document('file_name.txt'):
    chunks.append(chunk)

print(chunks)


['\x0cUncertainties in single-variable\nfunctions\nThe functional approach to obtain the uncertainty in a function Z = f (A),\nwhen A has been measured to be Ā ± α A :\n\x02 \x03\n\x04\n\x03 \x04\x02\nα Z = \x02 f Ā + α A − f Ā \x02 .\n\nZ = f (A)\nf (Ā + αA )\nf (Ā)\n\nThis is shown schematically in the ﬁgure.\nAssuming small uncertainties, such that the function can be approximated\nĀ\nĀ + αA\nas a straight line in the vicinity of Ā, then the calculus-based approximation to\nthis result is:\n\x02 \x02\nAn uncertainty α A in the variable A maps\n\x02 dZ \x02\ndirectly into an uncertainty α Z in the function\nα Z = \x02\x02 \x02\x02 α A .\ndA\nZ = f (A).\nTable 1 Results for the propagation of uncertainties in single-variable functions. The results for the trigonometric functions assume that the angles and\ntheir uncertainties are in radians.\nFunction, Z (A)\n\ndZ\ndA\n\n1\nA\nexp A\n\n−\n\n1\nA2\nexp A\n\nαz =\n\n1\nA\n\nαz =\n\nln A\n\nUncertainty\n\x02α \x02 \x02α \x02\nαA\

In [18]:
db = client.embeddings.create(
  model="text-embedding-ada-002",
  input=chunks,
  encoding_format="float"
)

In [19]:
query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query],
    encoding_format="float"
)

query_embedding = query_response.data[0].embedding
query_embedding = np.array(query_embedding).flatten()

print(query_embedding)

[-0.00207193  0.00108825 -0.01498739 ...  0.01438478 -0.0155971
 -0.04474949]


In [20]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

similarity_scores = []
for embedding_data in db.data:
    chunk_embedding = embedding_data.embedding

    chunk_embedding = np.array(chunk_embedding).flatten()

    score = cosine_similarity(query_embedding, chunk_embedding)
    similarity_scores.append(score)
    
print(similarity_scores)

[0.7122081257527494, 0.7208577527499321, 0.7579591474414539, 0.7282442784317867, 0.7227790455773949, 0.7579457943196682, 0.7858806253936579, 0.798717538399048, 0.7288407178337333, 0.7282808156737461, 0.7303737005114859, 0.7230495662122927, 0.7236369975655741, 0.7097042610082006, 0.7024222413288089, 0.7176272247511002, 0.7099194163872667, 0.7196004817619154, 0.7192680192700882, 0.7412721123479225, 0.7367121857681002, 0.7525936767736485, 0.7488726644729342, 0.7259114831581245, 0.7415241353157187, 0.7306609135930535, 0.7332544437848795, 0.7414185141408092, 0.7347839956685149, 0.7528838261623895, 0.7698288903898622, 0.7334626487897024, 0.727469988470392, 0.7594677029236606, 0.7425021613887174, 0.760804994474767, 0.7246162584776124, 0.7455520924772668, 0.7430166937126378, 0.7380924148751191, 0.7523425122935409, 0.7667261633720832, 0.746109842180755, 0.7369515681181859, 0.7366054695192764, 0.7190595305080234, 0.7339653699565157, 0.7274576869762434, 0.7458497513720624, 0.7472600520096788, 0.7

In [ ]:
chunk_scores = list(zip(chunks, similarity_scores))
sorted_chunks = sorted(chunk_scores, key=lambda x: x[1], reverse=True)

top_n = 5
combined_string = "All documents sourced from: file_name .txt"
for i in range(top_n):
    combined_string += sorted_chunks[i][0]
    
print(combined_string)

All documents sourced from: Measurements and their Uncertainties A practical guide to modern error analysis by I.G. Hughes and T.P.A.Hase .txt-squared statistic, χν2 ;
2
equal to the ﬁt value or
(2) the probability of obtaining a value of χmin
2
higher, given ν, P(χmin ; ν).
The χν2 statistic is signiﬁcantly easier to calculate and can be used to reject
the null hypothesis if χν2 > 3. Ambiguity arises in using the χν2 statistic to
reject the null hypothesis for values in the range 1 ≤ χν2 ≤ 3 due to a strong
dependence of the χν2 conﬁdence levels on ν as seen in Table 8.1. This
ambiguity can be resolved by calculating (using appropriate software or look2 or higher
up tables) the probability of obtaining the observed value of χmin
2
for the number of degrees of freedom, P(χmin ; ν). This probability has a
much weaker dependence on ν and the null hypothesis should be rejected if
2 ; ν) < 10−4 .
P(χmin
By rejecting the null hypothesis we are stating that the discrepancies
between the data

We've updated our prompt a little bit so that the chatbot knows how we want equations to be outputted

In [ ]:
prompt = f"Answer the question based only on this context: {combined_string}. Answer the question based on the above context: {query}, you should explain as if i do not have access to this context, always quote _______ as your source. run any code you create, quote explicitely any equations in latex format, always use $ when writing latex"

In [23]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": prompt,        
    }
  ]
)

Here we've changed some of the functions by first initialising in def __init__(self) a string refered to as buffer. This string is then updated in the on_text_created and on_text_delta functions where we have replaced them outputting text with them storing text instead. We then use the format_buffer function to format the text into something that latex can easily read, we essentially just split the plain text, from code, from latex math. The display_output function then takes the processed text from the format_buffer function and displays them as appropriate making the output of our chatbot more readable 

In [24]:
from typing_extensions import override
from openai import AssistantEventHandler
from PIL import Image
from IPython.display import display, Markdown, Latex, HTML
import io
import requests

class EventHandler(AssistantEventHandler):
    
    def __init__(self):
        super().__init__()
        self.buffer = ""  # Buffer to collect text output
        
    @override
    def on_text_created(self, text) -> None:
        if not self.buffer.endswith(text.value):  # Prevent duplication
            self.buffer += text.value

    @override
    def on_text_delta(self, delta, snapshot):
        if not self.buffer.endswith(delta.value):  # Prevent duplication
            self.buffer += delta.value

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)
            
    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)
                    elif output.type == "image":
                        # Fetch the image data using the file_id
                        file_id = output.image.file_id
                        image_data = self.download_image(file_id)
                        if image_data:
                            image = Image.open(io.BytesIO(image_data))
                            image.show()
  
    def download_image(self, file_id):
        url = f"https://api.openai.com/v1/files/{file_id}/content"
        headers = {
            "Authorization": f"Bearer {openai_api_key}",
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.content
        else:
            print(f"Failed to download image: {response.status_code} {response.text}")
            return None
        
    def display_output(self):

        # Process the buffer to format LaTeX and code blocks
        processed_content = self.format_buffer(self.buffer)

        # Display as Markdown to correctly render LaTeX and plain text
        display(Markdown(processed_content))

    def format_buffer(self, buffer):
        # Split buffer into lines for processing
        lines = buffer.split("\n")
        formatted_lines = []

        in_code_block = False

        for line in lines:
            if line.strip().startswith("code_interpreter"):
                in_code_block = True
                formatted_lines.append(line)
            elif line.strip() == "```":
                in_code_block = False
                formatted_lines.append(line)
            elif in_code_block:
                formatted_lines.append(line)
            else:
                # Check for LaTeX patterns and wrap them in delimiters
                line = line.replace(r'\(', '$').replace(r'\)', '$')
                line = line.replace(r'\[', '$$').replace(r'\]', '$$')
                formatted_lines.append(line)
        
        return "\n".join(formatted_lines)
            
    

Note here that when you run the stream, you need to 1: create a unique EventHandler(), 2: specify that EventHandler in the stream and 3: without event_handler.display_output() no visible output would be given, so this is an important step to be able to see your output.

In [25]:
event_handler = EventHandler()

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="",
  event_handler=event_handler,
) as stream:
  stream.until_done()

event_handler.display_output()

To find the minimum chi-squared value ($\chi_{\text{min}}^2$), you can follow a systematic procedure. This process is essential for testing the quality of a fit between your data and a theoretical model. Here’s a step-by-step guide:

1. **Obtain the Best-Fit Parameters**:
   - First, you need to determine the best-fit parameters for your model. These parameters are those that minimize the chi-squared value. The equations for obtaining these parameters depend on the specific model you are using. For a straight-line fit, you can use the equations provided in the book "Measurements and their Uncertainties: A practical guide to modern error analysis" by I.G. Hughes and T.P.A. Hase.

2. **Calculate the Normalized Residuals**:
   - For each data point, calculate the normalized residual. The normalized residual $R_i$ for a data point $y_i$ with an associated uncertainty $\alpha_i$ is given by:
     $$
     R_i = \frac{y_i - y(x_i)}{\alpha_i}
     $$
     Here, $y(x_i)$ is the value predicted by the best-fit function at the point $x_i$.

3. **Sum the Squares of the Normalized Residuals**:
   - The minimum chi-squared value ($\chi_{\text{min}}^2$) is obtained by summing the squares of the normalized residuals for all data points:
     $$
     \chi_{\text{min}}^2 = \sum_{i} R_i^2 = \sum_{i} \left(\frac{y_i - y(x_i)}{\alpha_i}\right)^2
     $$

### Example for a Straight-Line Fit

For a straight-line fit, the procedure is as follows:

1. **Determine the Best-Fit Parameters**:
   - Use the equations for a straight-line fit to find the slope and intercept that minimize $\chi^2$.

2. **Calculate the Normalized Residuals**:
   - For each data point $(x_i, y_i)$ with uncertainty $\alpha_i$, calculate:
     $$
     R_i = \frac{y_i - (mx_i + c)}{\alpha_i}
     $$
     where $m$ is the slope and $c$ is the intercept of the best-fit line.

3. **Sum the Squares of the Normalized Residuals**:
   - Compute the minimum chi-squared value:
     $$
     \chi_{\text{min}}^2 = \sum_{i} \left(\frac{y_i - (mx_i + c)}{\alpha_i}\right)^2
     $$

By following these steps, you can find the minimum chi-squared value, which is a crucial part of assessing the quality of your fit. This process is detailed in "Measurements and their Uncertainties: A practical guide to modern error analysis" by I.G. Hughes and T.P.A. Hase.

Similarly as we've done before you can then use the following two boxes to have a conversation, each time you use it you need only change the message in thread_message content= then run the next box to continue the conversation as we're use messages.create here to add messages to the already existing thread. Also note that we need to create a new EventHandler() everytime we want a new output as the current eventhandler only stores the most recent output, dont worry about storage though because we can simply just overwrite this eventhandler everytime we send a new message and then store the messages seperately. 

In [26]:
thread_message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="what chapter would i find it in again?"
)



In [27]:
event_handler2 = EventHandler()

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="",
  event_handler=event_handler2,
) as stream:
  stream.until_done()

event_handler2.display_output()

You can find the detailed procedure for obtaining the minimum chi-squared value ($\chi_{\text{min}}^2$) in Chapter 8 of "Measurements and their Uncertainties: A practical guide to modern error analysis" by I.G. Hughes and T.P.A. Hase. Specifically, the relevant sections are:

- **Section 8.4**: Using $\chi^2$ as a hypothesis test
- **Section 8.5**: Testing the quality of a fit using $\chi^2$

These sections provide a comprehensive explanation of how to calculate $\chi_{\text{min}}^2$ and use it to test the quality of a fit between your data and a theoretical model.